In [4]:
import yaml
from pydub import AudioSegment
import numpy as np
import wave
import os
import numpy as np
import soundfile as sf
from scipy.signal import stft, istft
import matplotlib.pyplot as plt
from tqdm import tqdm

/home/zaptos/miniconda3/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [16]:
directory = "data/babyslakh_16k"
listdir = os.listdir(directory)
listdir.sort()
tr_dicts = [{}]*len(listdir)
tr_dicts_2 = [{}]*len(listdir)

for i, tr in enumerate(tqdm(listdir)):
    tr_path = os.path.join(directory, tr)
    with open(os.path.join(tr_path, "metadata.yaml")) as meta:
        metadata = yaml.load(meta, Loader=yaml.Loader)
    # add Mix to the dictionary
    f, t, Zxx = stft(sf.read(os.path.join(directory,tr, 'mix'+format))[0], fs=16000, nperseg=256)
    tr_dicts_2.update({'mix': Zxx.copy()})
    file_inst = []
    for stem in metadata['stems'].keys():
        file_inst.append([stem + '.wav', metadata['stems'][stem]['inst_class']])
    file_inst = np.array(file_inst)
    file_inst = file_inst[[file_inst[:,0][i] in os.listdir(tr_path + '/stems') for i in range(len(file_inst))]]
    # combine all stems from the same instrument and track using soundfile
    for inst in np.unique(file_inst[:,1]):
        inst_files = file_inst[file_inst[:,1] == inst][:,0]
        inst_files = [os.path.join(tr_path, 'stems', inst_file) for inst_file in inst_files]
        inst_data = np.array([sf.read(inst_file)[0] for inst_file in inst_files])
        inst_data = np.sum(inst_data, axis=0)
        #update the dictionary with the combined data and the instrument name as the key
        tr_dicts[i].update({inst: inst_data})
        # compute the STFT of the combined data
        if inst in ['Bass', 'Guitar', 'Drums']:
            f, t, Zxx = stft(inst_data, fs=16000, nperseg=256)
            tr_dicts_2[i].update({inst: Zxx})


# Plot the magnitude of the STFT for each instrument
#for i, tr in enumerate(tqdm(tr_dicts_2)):
#    for inst in tr.keys():
#        plt.pcolormesh(t, f, tr[inst], shading='gouraud')
#        plt.colorbar(label='Magnitude')
#        plt.title(f'STFT Magnitude: Track {i} Instrument: ' + inst)
#        plt.xlabel('Time [s]')
#        plt.ylabel('Frequency [Hz]')
#        plt.show()

  0%|          | 0/20 [00:00<?, ?it/s]


TypeError: can only concatenate str (not "builtin_function_or_method") to str

In [7]:
import yaml
import numpy as np
import os
import soundfile as sf
from scipy.signal import stft
from tqdm import tqdm
import torch
# If you have a GPU, put the data on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

directory = "data/babyslakh_16k"
listdir = os.listdir(directory)
listdir.sort()
format = '.wav'
savedir = 'data/train'
sample_freq = 16000
freq_amount = 129
max_size = 1
listdir = listdir[:max_size]

In [8]:
for i, tr in enumerate(tqdm(listdir)):
    tr_dicts_2 = {}
    tr_path = os.path.join(directory, tr)
    with open(os.path.join(tr_path, "metadata.yaml")) as meta:
        metadata = yaml.load(meta, Loader=yaml.Loader)
    # add Mix to the dictionary
    f, t, Zxx = stft(sf.read(os.path.join(directory,tr, 'mix'+format))[0], fs=sample_freq, nperseg=freq_amount*2-2)
    tr_dicts_2.update({'mix': torch.tensor(Zxx.copy(), dtype=torch.complex128, device=device)})
    file_inst = []
    for stem in metadata['stems'].keys():
        file_inst.append([stem + format, metadata['stems'][stem]['inst_class']])
    file_inst = np.array(file_inst)
    file_inst = file_inst[[file_inst[:,0][i] in os.listdir(tr_path + '/stems') for i in range(len(file_inst))]]
    # combine all stems from the same instrument and track using soundfile
    for inst in np.unique(file_inst[:,1]):
        inst_files = file_inst[file_inst[:,1] == inst][:,0]
        inst_files = [os.path.join(tr_path, 'stems', inst_file) for inst_file in inst_files]
        inst_data = np.array([sf.read(inst_file)[0] for inst_file in inst_files])
        inst_data = np.sum(inst_data, axis=0)
        # compute the STFT of the combined data
        if inst in ['mix','Bass', 'Guitar', 'Drums', 'Piano']:
            f, t, Zxx = stft(inst_data, fs=sample_freq, nperseg=freq_amount*2-2)
            tr_dicts_2.update({inst: torch.tensor(Zxx.copy(), dtype=torch.complex128, device=device)})
    # save the dictionary as a .pt file
    torch.save(tr_dicts_2, os.path.join(savedir, tr + '.pt'))
    torch.cuda.empty_cache()

100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


In [12]:
t, f = istft((tr_dicts_2['mix'].cpu().numpy()), fs=sample_freq)

In [16]:
sf.write('test.wav', f, 8000)